This notebook is a driver program for _taxes_ library

In [55]:
%load_ext autoreload
%autoreload 2

from taxes.download import get_sheet_links_names, download_sheet_series, get_gus_stats
import pandas as pd
import pathlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# getting all sheet links 
year = 2020
sheets = get_sheet_links_names(year) 

dir_sheets = download_sheet_series(sheets)

print(sheets, dir_sheets)

(['https://www.gov.pl/attachment/31d60032-a3c5-4e4f-8af8-67c8fa09afd2', 'https://www.gov.pl/attachment/82cb06d7-02e6-4d24-a8b4-9926fe0a3079', 'https://www.gov.pl/attachment/bafb6020-bca0-4ec8-9369-845e0afb94d9', 'https://www.gov.pl/attachment/e4077a76-1fbc-478e-a15d-eea0a4e3f130', 'https://www.gov.pl/attachment/0b98f8be-e9e1-48e3-8bc5-796e8c0b169e'], ['20210215_Gminy_2_za_2020.xlsx', '20210211_Powiaty_za_2020.xlsx', '20210215_Miasta_NPP_2_za_2020.xlsx', '20210211_Metropolia_2020.xlsx', '20210211_Województwa_za_2020.xlsx']) 5
20210215_Gminy_2_za_2020.xlsx downloaded!


20210211_Powiaty_za_2020.xlsx downloaded!


20210215_Miasta_NPP_2_za_2020.xlsx downloaded!


Downloaded webpage. Switching to mirror source...
20210211_Metropolia_2020.xlsx downloaded!


Downloaded webpage. Switching to mirror source...
20210211_Województwa_za_2020.xlsx downloaded!


All sheets downloaded!

(['https://www.gov.pl/attachment/31d60032-a3c5-4e4f-8af8-67c8fa09afd2', 'https://www.gov.pl/attachment/82cb06d7-02e6

In [15]:
# fl = pathlib.Path("/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Gminy_za_2019.xlsx")
fl = dir_sheets.joinpath(sheets[1][2])
print(fl)

col_names = ["WK", "PK", "GK", "GT", "Nazwa JST", "województwo", "powiat", "DZIAŁ", "ROZDZIAŁ", "PARAGRAF", 
             "Należności (saldo początkowe plus przypisy minus odpisy)", "Dochody wykonane (wpłaty minus zwroty)", 
             "ogółem", "zaległości netto", "nadpłaty"]
# df = pd.read_excel(str(fl), skiprows=7, header=None, names=col_names, engine='openpyxl')
file_extension = fl.suffix.lower()[1:]

if file_extension == 'xlsx':
    df = pd.read_excel(fl, skiprows=7, header=None, names=col_names, engine='openpyxl')
elif file_extension == 'xls':
    df = pd.read_excel(fl, skiprows=7, header=None, names=col_names, engine='xlrd')
else:
    raise Exception("File not supported")
    
df

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Miasta_NPP_za_2019.xlsx


,WK,PK,GK,GT,Nazwa JST,województwo,powiat,DZIAŁ,ROZDZIAŁ,PARAGRAF,Należności (saldo początkowe plus przypisy minus odpisy),Dochody wykonane (wpłaty minus zwroty),ogółem,zaległości netto,nadpłaty
0,2,61,-,-,Jelenia Góra,dolnośląskie,-,756,75621,10,84424413,84411158,13255,13255,NaN
1,2,61,-,-,Jelenia Góra,dolnośląskie,-,756,75622,10,22724531,22720957,3574,3574,NaN
2,2,62,-,-,Legnica,dolnośląskie,-,756,75621,10,112627141,112609457,17684,17684,NaN
3,2,62,-,-,Legnica,dolnośląskie,-,756,75622,10,30315863,30311095,4768,4768,NaN
4,2,64,-,-,Wrocław,dolnośląskie,-,756,75621,10,1161913172,1161730741,182431,182431,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,32,61,-,-,Koszalin,zachodniopomorskie,-,756,75622,10,35651058,35645450,5608,5608,NaN
128,32,62,-,-,Szczecin,zachodniopomorskie,-,756,75621,10,503933861,503854738,79123,79123,NaN
129,32,62,-,-,Szczecin,zachodniopomorskie,-,756,75622,10,135643951,135622616,21335,21335,NaN
130,32,63,-,-,Świnoujście,zachodniopomorskie,-,756,75621,10,43593627,43586782,6845,6845,NaN


In [ ]:
# df_1 = df[df["Nazwa JST"] == "BOLESŁAWIEC"]
# df_1 = df_1[df_1["województwo"] == "dolnośląskie"]
# df_1

In [8]:
df

,WK,PK,GK,GT,Nazwa JST,województwo,powiat,DZIAŁ,ROZDZIAŁ,PARAGRAF,Należności (saldo początkowe plus przypisy minus odpisy),Dochody wykonane (wpłaty minus zwroty),ogółem,zaległości netto,nadpłaty
0,2,1,-,-,bolesławiecki,dolnośląskie,-,756,75622,10,21984822,21981364,3458,3458,NaN
1,2,2,-,-,dzierżoniowski,dolnośląskie,-,756,75622,10,21906718,21903272,3446,3446,NaN
2,2,3,-,-,głogowski,dolnośląskie,-,756,75622,10,30347660,30342887,4773,4773,NaN
3,2,4,-,-,górowski,dolnośląskie,-,756,75622,10,5784270,5783360,910,910,NaN
4,2,5,-,-,jaworski,dolnośląskie,-,756,75622,10,10550026,10548367,1659,1659,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,32,14,-,-,stargardzki,zachodniopomorskie,-,756,75622,10,27896289,27891901,4388,4388,NaN
310,32,15,-,-,szczecinecki,zachodniopomorskie,-,756,75622,10,15901267,15898766,2501,2501,NaN
311,32,16,-,-,świdwiński,zachodniopomorskie,-,756,75622,10,8990457,8989043,1414,1414,NaN
312,32,17,-,-,wałecki,zachodniopomorskie,-,756,75622,10,11274515,11272742,1773,1773,NaN


In [60]:
get_gus_stats()

PosixPath('/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/gus')

In [67]:
!pytest 

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-7.0.0, pluggy-1.0.0
rootdir: /Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final
plugins: anyio-3.5.0
collected 5 items                                                              

taxes/tests/test_download.py .....                                       [100%]

=============================== warnings summary ===============================
../../../../../../miniforge3/lib/python3.9/site-packages/_pytest/config/__init__.py:716
  /Users/k4cp3rskiii/miniforge3/lib/python3.9/site-packages/_pytest/config/__init__.py:716: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: taxes.download
    self.import_plugin(import_spec)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 5 passed, 1 warning in 1.32s =========================


In [68]:
!pre-commit run --all --show-diff-on-failure

black....................................................................Passed
flake8...................................................................Passed
Check JSON...........................................(no files to check)Skipped
Check Yaml...............................................................Passed
Check for merge conflicts................................................Passed
Fix End of Files.........................................................Passed
Trim Trailing Whitespace.................................................Passed
Debug Statements (Python)................................................Passed
Mixed line ending........................................................Passed
setup-cfg-fmt............................................................Passed
pyupgrade................................................................Passed
rst ``code`` is two backticks........................(no files to check)Skipped
isort...................................